In [1]:
!pip install igraph

In [3]:
import networkx as nx
import igraph as ig

In [39]:
import numpy as np
import itertools

In [26]:
from IPython.display import display, HTML

In [14]:
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

In [31]:


def animate_graphs(graphs):
    fig, ax = plt.subplots()
    
    def update(i):
        ax.clear()
        nx.draw(graphs[i], ax=ax)
    
    anim = FuncAnimation(fig, update, frames=len(graphs), interval=100)
    display(HTML(anim.to_jshtml()))



## Descriptive statistics

In [117]:
motifs = {
    '2-star': nx.Graph([(1,2),(2,3)]),
    'Triangle': nx.Graph([(1,2),(1,3),(2,3)]),
    'Square': nx.Graph([(1,2),(2,3),(3,4),(1,4)]),
    'Square_one_diag': nx.Graph([(1,2),(3,2),(3,4), (1,4), (1,3)])
    }



def calculate_closeness(G):
    closeness_centrality = {}
    for node in G.nodes():
        closeness_centrality[node] = nx.closeness_centrality(G, node, wf_improved = False)
    return closeness_centrality

def calculate_centrality(G):
    return nx.degree_centrality(G)

def calculate_betweenness(G):
    return nx.betweenness_centrality(G)

def calculate_motif_count(G, mo):
    mcount = dict(zip(mo.keys(), list(map(int, np.zeros(len(mo))))))
    nodes = G.nodes()

    triplets = list(itertools.product(*[nodes, nodes, nodes]))
    quadralets = list(itertools.product(*[nodes, nodes, nodes, nodes]))
    triplets = [trip for trip in triplets if len(list(set(trip))) == 3]
    quadralets = [quad for quad in quadralets if len(list(set(quad))) == 4]
    triplets = map(list, map(np.sort, triplets))
    u_triplets = []

    for trip in triplets:
        if u_triplets.count(trip)== 0:
            u_triplets.append(trip)

    quadralets = map(list, map(np.sort, quadralets))
    for quad in quadralets:
        if u_quadralets.count(quad)== 0:
            u_quadralets.append(quad)
    three_node_mo = list(mo.keys())[0:2]
    four_node_mo = list(mo.keys())[2:4]


    for trip in u_triplets:
        sub_gr = G.subgraph(trip)
        mot_match = list(map(lambda mot_id: nx.is_isomorphic(sub_gr, mo[mot_id]), three_node_mo))
        match_keys = [three_node_mo[i] for i in range(2) if mot_match[i]]
        if len(match_keys) == 1:
            mcount[match_keys[0]] += 1
    for quad in u_quadralets:
        sub_gr = G.subgraph(quad)
        mot_match = list(map(lambda mot_id: nx.is_isomorphic(sub_gr, mo[mot_id]), four_node_mo))
        match_keys = [four_node_mo[i] for i in range(2) if mot_match[i]]
        if len(match_keys) == 1:
            mcount[match_keys[0]] += 1
    return mcount



## Simulating Erdos Renyi Random Graphs

In [141]:
graphs = []
for i in range(20):
    G = nx.erdos_renyi_graph(5, np.random.uniform())
    graphs.append(G)

# Animate the graphs

#animate_graphs(graphs)

In [142]:
closeness_centrality = calculate_closeness(G)
degree_centrality = calculate_centrality(G)
betweenness_centrality = calculate_betweenness(G)
motif_count = calculate_motif_count(G, motifs)

print("Closeness centrality:", closeness_centrality)
print("Degree centrality:", degree_centrality)
print("Betweenness centrality:", betweenness_centrality)
print("Motif count:", motif_count)


Closeness centrality: {0: 0.8, 1: 0.8, 2: 0.8, 3: 0.5, 4: 0.5714285714285714}
Degree centrality: {0: 0.75, 1: 0.75, 2: 0.75, 3: 0.25, 4: 0.5}
Betweenness centrality: {0: 0.5, 1: 0.16666666666666666, 2: 0.16666666666666666, 3: 0.0, 4: 0.0}
Motif count: {'2-star': 4, 'Triangle': 2, 'Square': 0, 'Square_one_diag': 1}
